<a href="https://colab.research.google.com/github/shinnew9/Apziva_practice_code/blob/main/Project3-PotentialTalents/(mergedcsv)OLMoE_LLMFineTuning%2BLoRA%2BRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

While OpenAI provides fine-tuning APIs, but GPT-4 does not support fine-tuning yet. I can fine-tune GPT-3.5-turbo instead but since the model is quite outdated, I will choose to finetune the latest LLM downloaded from HuggingFace.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install Libraries for LLM finetuning

In [ ]:
# !pip install deepspeed

In [ ]:
!pip install transformers peft accelerate datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 70.8 MB/s eta 0:00:00


### Open CSV

In [ ]:
import pandas as pd
import json

df = pd.read_csv("/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data.csv")
df_copy = df.copy()
df_copy

df2 = pd.read_excel("/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data2.xlsx")
df2_copy = df2.copy()
df2_copy

,id,title,location,screening_score
0,1,innovative and driven professional seeking a r...,United States,100
1,2,ms applied data science student usc research a...,United States,100
2,3,computer science student seeking full-time sof...,United States,100
3,4,microsoft certified power bi data analyst mba ...,United States,100
4,5,graduate research assistant at uab masters in ...,United States,100
...,...,...,...,...
1280,1281,GenAINLPCVML Senior AI Engineer Data Scientist...,India,0
1281,1282,Data Scientist and Analyst Driving Business In...,Kenya,0
1282,1283,Software Engineer WriterSpace Science Enthusiast,Ethiopia,0
1283,1284,Master Trainer of Big Data Analytics Data Scie...,Jordan,0


In [ ]:
df_merged = pd.concat([df_copy, df2_copy], ignore_index=True)
df_merged

,id,job_title,location,connection,fit,title,screening_score
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN,NaN,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN,NaN,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,NaN,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,NaN,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1384,1281,NaN,India,NaN,NaN,GenAINLPCVML Senior AI Engineer Data Scientist...,0.0
1385,1282,NaN,Kenya,NaN,NaN,Data Scientist and Analyst Driving Business In...,0.0
1386,1283,NaN,Ethiopia,NaN,NaN,Software Engineer WriterSpace Science Enthusiast,0.0
1387,1284,NaN,Jordan,NaN,NaN,Master Trainer of Big Data Analytics Data Scie...,0.0


In [ ]:
# convert csv to jsonl

jsonl_data = df_merged.apply(lambda x: json.dumps({
    "prompt": f"Job Title: {x['job_title']}\nLocation: {x['location']}\nConnections: {x['connection']}",
    "completion": str(x['fit'])
}), axis=1)

# Save as JSONL file
with open("/content/drive/MyDrive/Apziva/3rd_PotentialTalents/latestmodels/train_data2.jsonl", "w") as f:
  f.write("\n".join(jsonl_data))

print("Training data saves as train_data.jsonl")

print(jsonl_data)

Training data saves as train_data.jsonl
0       {"prompt": "Job Title: 2019 C.T. Bauer College...
1       {"prompt": "Job Title: Native English Teacher ...
2       {"prompt": "Job Title: Aspiring Human Resource...
3       {"prompt": "Job Title: People Development Coor...
4       {"prompt": "Job Title: Advisory Board Member a...
                              ...                        
1384    {"prompt": "Job Title: nan\nLocation: India\nC...
1385    {"prompt": "Job Title: nan\nLocation: Kenya\nC...
1386    {"prompt": "Job Title: nan\nLocation: Ethiopia...
1387    {"prompt": "Job Title: nan\nLocation: Jordan\n...
1388    {"prompt": "Job Title: nan\nLocation: Pakistan...
Length: 1389, dtype: object


In [ ]:
# In case I might use more

### Fine-Tuning Open-Source Models (HuggingFace+Lora)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


# Load Model & Tokenizer
# I applied Qwen/Qwen2.5-Math-1.5b was large enough to run in my environment, so didn't even think of trying a larger model, like Qwen/Qwen2.5-Math-7B
# Load Mistral in 4-bit Mode to Save Memory
model_name = "allenai/OLMoE-1B-7B-0125-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,   # Load model in 4-bit
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type = "nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = "auto"
)

# Apply LoRA Configuration
lora_config = LoraConfig(
    r = 8,                  # LoRA rank
    lora_alpha = 16,        # LoRA scaling factor
    target_modules = ["q_proj", "v_proj"],
    lora_dropout = 0.05,    # Dropout probability
    bias = "none"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model = prepare_model_for_kbit_training(model)

print("OLMo with LoRA is ready!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

OLMo with LoRA is ready!


In [ ]:
from datasets import load_dataset

dataset = load_dataset('json', data_files= '/content/drive/MyDrive/Apziva/3rd_PotentialTalents/latestmodels/train_data2.jsonl')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def tokenize(example):
    # Prompt + Completion을 하나로 합쳐서 처리
    full_prompt = example["prompt"] + "\nFit Score: " + example["completion"]
    tokenized = tokenizer(
        full_prompt,
        padding="max_length",
        truncation = True,
        max_length = 128
        # return_tensors = "pt"
    )

    # labels도 input_ids와 동일하세 설정 (causal LM 방식)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized


tokenized_dataset = dataset["train"].map(
    tokenize,
    remove_columns = dataset["train"].column_names
)

Map:   0%|          | 0/1389 [00:00<?, ? examples/s]

### Training

In [ ]:
from transformers import Trainer, TrainingArguments
import json

# Fine-tuning arguments
training_args = TrainingArguments(
      output_dir = "/content/drive/MyDrive/Apziva/3rd_PotentialTalents/latestmodels/OLMoE-1B-Finetuned/results",
      num_train_epochs = 3,  # Number of training epochs
      per_device_train_batch_size = 2,  # Adjust based on my GPU memory
      save_strategy = "epoch",
      logging_dir = "/content/drive/MyDrive/Apziva/3rd_PotentialTalents/latestmodels/OLMoE-1B-Finetuned/logs",
      remove_unused_columns = False
)

# Trainer setup
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)


# Start training
trainer.train()

<ipython-input-10-a32d083c9c02>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yoojinshin9918 (Apziva-Project3) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
500,17.557800
1000,17.598700
1500,17.565000
2000,17.568900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 854666bf-e59d-41d9-8337-4a5d654d2dc5)') - silently ignoring the lookup for the file config.json in allenai/OLMoE-1B-7B-0125-Instruct.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in a

TrainOutput(global_step=2085, training_loss=17.57299143107389, metrics={'train_runtime': 4865.0907, 'train_samples_per_second': 0.857, 'train_steps_per_second': 0.429, 'total_flos': 2.181674628808704e+16, 'train_loss': 17.57299143107389, 'epoch': 3.0})

### Inference

In [ ]:
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/Apziva/3rd_PotentialTalents/latestmodels/OLMoE-1B-Finetuned/results/checkpoint-156"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code = True,
    low_cpu_mem_usage = True)  # 중요

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# fit score만 뽑는 함수
def get_fit_score(job_title, search_term):
    prompt = f"""
    Given the job title and search term, assign a numerical fit score between 0 and 1 based on their similarity.
    The fit score should be a single number with no explanation.

    Job Title: {job_title}
    Search Term: {search_term}

    Fit Score:
    """

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    #.to("cuda")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,     # deterministic output
            temperature=0.0,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    return extract_fit_score(decoded)

    print("🧾 Decoded Output:\n", decoded)

In [ ]:
import re

def extract_fit_score(text):
  matches = re.findall(r"\b([0-1]\.\d+)\b", text)
  if matches:
      score = float(matches[0])
      return max(0.0, min(1.0, score))
  else:
    print(f"[Warning]Could not extract score from: {text}")
    return 0.0

In [ ]:
print(decoded)

NameError: name 'decoded' is not defined

In [ ]:
decoded = tokenizer.decode(output[0], skip_special_tokens =True)

NameError: name 'output' is not defined

In [ ]:
print(f"[DEBUG] Decoded Output: \n{decoded}")

NameError: name 'decoded' is not defined

In [ ]:
# Extracting only number
# def extract_fit_score(text):
#     match = re.search(r"([0-1]\.\d+)", text)
#     return float(match.group(1)) if match else 0.0

# # Testing
# job_title = "HR Manager"
# search_term = "Aspiring Human Resources"
# score = get_fit_score(job_title, search_term)
# print(f"Predicted Fit Score: {score}")

In [ ]:
from tqdm.notebook import tqdm  # for Jupyter/Colab
tqdm.pandas()

search_term = "Aspiring Human Resources"  # example, you may change it


if "fit_score" in df_merged.columns:
    del df_merged["fit_score"]


df_merged["fit_score"] = df_merged.progress_apply(
    lambda row: get_fit_score(row["job_title"], search_term),
    axis=1
)


# 높은 fit_score 순으로 정렬 및 rank 부여
df_merged["rank"] = df_merged["fit_score"].rank(ascending=False, method="min").astype(int)
df_merged_sorted = df_merged.sort_values(by="fit_score", ascending=False).reset_index(drop=True)

# 결과 확인
df_merged_sorted[["job_title", "location", "connection", "fit_score", "rank"]].head(10)

  0%|          | 0/1389 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[Warning]Could not extract score from: 
    Given the job title and search term, assign a numerical fit score between 0 and 1 based on their similarity.
    The fit score should be a single number with no explanation.

    Job Title: Human Resources|
Conflict Management|
Policies & Procedures|Talent Management|Benefits & Compensation
    Search Term: Aspiring Human Resources

    Fit Score:
    """
    def calculate_fit_score(job_title, search_term):
        


,job_title,location,connection,fit_score,rank
0,NaN,Pakistan,NaN,0.0,1
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.0,1
2,Native English Teacher at EPIK (English Progra...,Kanada,500+,0.0,1
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.0,1
4,People Development Coordinator at Ryan,"Denton, Texas",500+,0.0,1
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0.0,1
6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.0,1
7,Student at Humber College and Aspiring Human R...,Kanada,61,0.0,1
8,HR Senior Specialist,San Francisco Bay Area,500+,0.0,1
9,Student at Humber College and Aspiring Human R...,Kanada,61,0.0,1


In [ ]:
# print(get_fit_score("Software Engineer", "Aspiring Human Resources"))  # 0.0
# print(get_fit_score("Recruiter", "Aspiring Human Resources"))   # 0.0
# print(get_fit_score("AI Researcher", "Aspiring Human Resources"))  # 0.0

# print(get_fit_score("Barista", "Aspiring Human Resources"))   # 0.0
# print(get_fit_score("Talent Acquisition Lead", "Aspiring Human Resources"))  # 예상: 0.9, 0.9
# print(get_fit_score("Senior Engineer", "Aspiring Human Resources"))  # 예상: 0.2~0.4, 0.9
# print(get_fit_score("UX Designer", "Aspiring Human Resources"))  # 예상: 0.5
# print(get_fit_score("Marketing Manager", "Aspiring Human Resources"))  # 0.0

print(get_fit_score("HR Specialist", "Aspiring Human Resources"))  # 0.0

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


0.0


### RAG

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_path = "/content/drive/MyDrive/Apziva/3rd_PotentialTalents/latestmodels/OLMoE-1B-Finetuned/results/checkpoint-156"
tokenizer = AutoTokenizer.from_pretrained(model_path)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 또는 8bit도 가능
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
    )

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)


llm = HuggingFacePipeline(pipeline=pipe)

# 3. Load your reference document (can be job description, resume corpus, etc.)
loader = TextLoader("/content/drive/MyDrive/Apziva/3rd_PotentialTalents/latestmodels/hr_reference.txt")
docs = loader.load()

# 4. Split into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 5. Embed documents using HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(splits, embedding_model)

# 6. RAG: RetrievalQA chain with your fine-tuned model
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# 7. Query your RAG system
query = "How relevant is a candidate with the title 'HR Business Partner' for 'Aspiring Human Resources'?"
result = rag_chain({"query": query})

print("🔍 RAG Response:\n", result['result'])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


🔍 RAG Response:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The position 'Aspiring Human Resources' expects someone motivated to grow into a full-time HR professional. Preferred background includes psychology, communication, or business management.

An ideal Human Resources candidate should demonstrate strong interpersonal skills, experience with talent acquisition and employee relations, and a deep understanding of organizational behavior. Familiarity with labor laws, training development, and conflict resolution is a plus.

Question: How relevant is a candidate with the title 'HR Business Partner' for 'Aspiring Human Resources'?
Helpful Answer:

Not very relevant.

The title 'HR Business Partner' is more focused on the financial and operational aspects of HR, including payroll, benefits administration, and performance management. It does not necessarily require the